In [42]:
import os
import numpy as np
import json

### Check if histograms share the same energy

In [28]:
def read_results_file(path):

    with open(path, 'r') as file:
        content = file.read()

    content = content.strip().rstrip(',')

    corrected_json = f'[{content}]'

    try:
        data = json.loads(corrected_json)
    except json.JSONDecodeError as e:
        print(f"Failed to parse JSON: {e}")

    return data

def has_duplicates(input_list):
    seen = set()
    for item in input_list:
        if item in seen:
            return True  # Found a duplicate
        seen.add(item)
    return False  # No duplicates found


In [29]:
filename = f"../results/periodic/prob_0.100000/X_4_Y_4/error_class_I/StitchedHistogram_intervals_10_iterations_1000_overlap_0.250000_walkers_8_alpha_0.800000_beta_0.0000010000_exchange_offset50.txt"
results = read_results_file(filename)

for seed in range(1000, 2000):

    # Prerun results
    histogram_energy = []

    with open(f"../init/periodic/prob_0.100000/X_4_Y_4/error_class_I/seed_{seed}/histogram/histogram.txt", "r") as file:
        for line in file:
            parts = line.split()
            if int(parts[1]) == 1:
                histogram_energy.append(int(parts[0]))

    for res in results:
        if int(res["histogram_seed"]) == seed:
            keys_as_int = list(map(int, res["results"].keys()))
            break

    if keys_as_int != histogram_energy:
        print(seed)



### Check duplicates

In [32]:
filename = f"../results/periodic/prob_0.100000/X_4_Y_4/error_class_I/StitchedHistogram_intervals_10_iterations_1000_overlap_0.250000_walkers_8_alpha_0.800000_beta_0.0000010000_exchange_offset50.txt"
results = read_results_file(filename)

for res in results:
    keys_as_int = list(map(int, res["results"].keys()))
    if has_duplicates(keys_as_int):
        print(res)

### Reproducability

In [33]:
filename = f"../results/periodic/prob_0.100000/X_4_Y_4/error_class_I/StitchedHistogram_intervals_10_iterations_1000_overlap_0.250000_walkers_8_alpha_0.800000_beta_0.0000010000_exchange_offset50.txt"
results = read_results_file(filename)

filename_reproduce = f"../results/periodic_reproduce/prob_0.100000/X_4_Y_4/error_class_I/StitchedHistogram_intervals_10_iterations_1000_overlap_0.250000_walkers_8_alpha_0.800000_beta_0.0000010000_exchange_offset50.txt"
results_reproduce = read_results_file(filename_reproduce)

In [44]:
diffs = []

for res in results:
    hist_seed = res['histogram_seed']
    log_g_result = res['results']

    for rep in results_reproduce:
        if rep['histogram_seed'] == hist_seed:
            results_rep = rep['results']
            diff = [log_g_result[key] - results_rep[key] for key in log_g_result]
            diffs.append(np.max(np.abs(diff)))